In [1]:
import os
import pandas as pd
import numpy as np
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
from google.datalab import storage
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
print("Hello")

Hello


In [3]:
#CONSTANTS
MANIFEST_PATH = 'gs://aave-test-bucket/DCB_manifest.csv'
WAV_FOLDER = 'data_processed_DCB/wav'
OUTPUT_PATH = 'DCB_manifest_transcribed.csv'

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="speech-rec-key.json"

In [5]:
root = "gs://aave-test-bucket"
audio_files = [o.key for o in storage.Bucket('aave-test-bucket').objects() if o.key.startswith(WAV_FOLDER)]
manifest = pd.read_csv(MANIFEST_PATH)
# manifest["google_transcription"] = np.nan

In [6]:
print(len(audio_files))

4361


In [7]:
manifest.head()

,wav_file,txt_file,groundtruth_text,duration,age,gender
0,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,"Uh, I want to- I'm going to pursue IT. And I'm...",13.180,18,Male
1,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,showing that all young black men shouldn't go ...,21.988,18,Male
2,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,[And one-] And once you in college we will try...,5.134,18,Male
3,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,"Uh, well, basically as of today, Congress Heig...",30.588,18,Male
4,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,"Uh, I had /began before the street life/ for- ...",28.943,18,Male


In [8]:
def get_single_transcript(storage_uri):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()

    # The language of the supplied audio
    language_code = "en-US"

    config = {
        "language_code": language_code,
    }
    
    audio = {"uri": storage_uri}
    try:

        operation = client.long_running_recognize(config, audio)

        response = operation.result()
    
        transcript = []
        for result in response.results:
            # First alternative is the most probable result
            alternative = result.alternatives[0]
            transcript.append(alternative.transcript.strip())

        return " ".join(transcript)
    except:
        print("CONNECTION FAILED")
        print(storage_uri)
        return np.nan
        

In [9]:
client = speech_v1.SpeechClient()

def get_single_transcript_multithreaded(audio_file):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    # The language of the supplied audio
    storage_uri = os.path.join(root, audio_file)
    language_code = "en-US"

    config = {
        "language_code": language_code,
    }
    
    audio = {"uri": storage_uri}
    try:

        operation = client.long_running_recognize(config, audio)

        response = operation.result()
    
        transcript = []
        for result in response.results:
            # First alternative is the most probable result
            alternative = result.alternatives[0]
            transcript.append(alternative.transcript.strip())

        return audio_file," ".join(transcript)
    except:
        print("CONNECTION FAILED")
        return audio_file,np.nan
        

In [11]:
#single process version
num = 10#len(audio_files)
for idx in tqdm(range(num)):
    print("iter")
    audio_file = audio_files[idx]
    full_file_path = os.path.join(root, audio_file)
    transcript = get_single_transcript(full_file_path)
    print(audio_file)
    print(transcript)
#     manifest.loc[manifest['wav_file'] == audio_file, "google_transcription"] = transcript
#     manifest.to_csv("ROC_manifest_transcribed.csv")

  0%|          | 0/10 [00:00<?, ?it/s]

iter


 10%|█         | 1/10 [00:04<00:44,  4.91s/it]

data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part_1.wav
I want it going to pursue it nonprofit program call big bro
iter


 20%|██        | 2/10 [00:07<00:32,  4.11s/it]

...

data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part_14.wav
if you go out with somebody and y'all get into it or if you more successful than we have to watch out maybe they might be trying to rob you with a probably try to kill you and stuff like that like I don't think that's likely going to break everybody want the same goal but nobody was trying to reach
iter


 90%|█████████ | 9/10 [01:12<00:11, 11.96s/it]

data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part_15.wav
I guess Parkland is beefing with a hood that's literally right down the street 22nd it's literally has the Boys & Girls Club in these row houses townhouses other than that it's basically nothing to separate
iter


100%|██████████| 10/10 [01:23<00:00,  8.30s/it]

data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part_16.wav
is Ali you can get junk it is shot at 4 you get if you get robbed if there's a lot of stuff, I got her people that kidnapped just Allah


In [13]:
%%time
num_audio_files = len(audio_files)
transcript_dict = {'audio_file':[], 'google_transcription': []}
with Pool(20) as p:
    with tqdm(total=num_audio_files) as prog_bar:
        for i, result in enumerate(tqdm(p.imap_unordered(get_single_transcript_multithreaded, audio_files))):
            prog_bar.update()
            transcript_dict['audio_file'].append(result[0])
            transcript_dict['google_transcription'].append(result[1])

  4%|▍         | 195/4361 [01:27<29:13,  2.38it/s]


  9%|▉         | 392/4361 [03:02<37:13,  1.78it/s]


 14%|█▎        | 597/4361 [04:37<15:31,  4.04it/s]


 18%|█▊        | 786/4361 [07:02<32:45,  1.82it/s]


 23%|██▎       | 987/4361 [08:26<26:47,  2.10it/s]


...

 88%|████████▊ | 3818/4361 [33:53<02:50,  3.18it/s]


 92%|█████████▏| 4020/4361 [35:33<02:12,  2.58it/s]


 96%|█████████▋| 4204/4361 [36:58<01:05,  2.38it/s]


100%|██████████| 4361/4361 [39:02<00:00,  4.21s/it]
4361it [39:02,  1.86it/s]
100%|██████████| 4361/4361 [39:02<00:00,  1.86it/s]


CPU times: user 15.3 s, sys: 4.82 s, total: 20.1 s
Wall time: 39min 3s


In [14]:
transcript_df = pd.DataFrame(transcript_dict)
print(len(transcript_df))
transcript_df.head()

4361


,audio_file,google_transcription
0,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,I just want to do everything
1,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,ugly one message for people made
2,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,I had a you practice and I just told me up for...
3,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,find something that you like by something is d...
4,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,because my family's most be spread out to dece...


In [15]:
print("HEEEELLLLOO")

HEEEELLLLOO


In [16]:
#join dataframes
manifest['key'] = manifest.wav_file.apply(lambda x: x.split("/")[-1])
manifest.head()

,wav_file,txt_file,groundtruth_text,duration,age,gender,key
0,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,"Uh, I want to- I'm going to pursue IT. And I'm...",13.180,18,Male,DCB_se1_ag1_m_01_1_part_1.wav
1,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,showing that all young black men shouldn't go ...,21.988,18,Male,DCB_se1_ag1_m_01_1_part_2.wav
2,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,[And one-] And once you in college we will try...,5.134,18,Male,DCB_se1_ag1_m_01_1_part_3.wav
3,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,"Uh, well, basically as of today, Congress Heig...",30.588,18,Male,DCB_se1_ag1_m_01_1_part_4.wav
4,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,"Uh, I had /began before the street life/ for- ...",28.943,18,Male,DCB_se1_ag1_m_01_1_part_5.wav


In [17]:
transcript_df['key'] = transcript_df.audio_file.apply(lambda x: x.split("/")[-1])
transcript_df.head()

,audio_file,google_transcription,key
0,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,I just want to do everything,DCB_se1_ag1_m_01_1_part_10.wav
1,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,ugly one message for people made,DCB_se1_ag1_m_01_1_part_100.wav
2,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,I had a you practice and I just told me up for...,DCB_se1_ag1_m_01_1_part_21.wav
3,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,find something that you like by something is d...,DCB_se1_ag1_m_01_1_part_101.wav
4,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,because my family's most be spread out to dece...,DCB_se1_ag1_m_01_1_part_12.wav


In [18]:
result = manifest.set_index('key').join(transcript_df.set_index('key'), how = 'inner',lsuffix='_caller', rsuffix='_other')
print(len(result))
result.head()

4361


,wav_file,txt_file,groundtruth_text,duration,age,gender,audio_file,google_transcription
key,,,,,,,,
DCB_se1_ag1_m_01_1_part_1.wav,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,"Uh, I want to- I'm going to pursue IT. And I'm...",13.180,18,Male,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,I want it going to pursue it nonprofit program...
DCB_se1_ag1_m_01_1_part_2.wav,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,showing that all young black men shouldn't go ...,21.988,18,Male,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,all young black man shouldn't go to the street...
DCB_se1_ag1_m_01_1_part_3.wav,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,[And one-] And once you in college we will try...,5.134,18,Male,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,don't want you in college me try to do college
DCB_se1_ag1_m_01_1_part_4.wav,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,"Uh, well, basically as of today, Congress Heig...",30.588,18,Male,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,baked brie as of today has lost in a month hou...
DCB_se1_ag1_m_01_1_part_5.wav,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,data_processed_DCB/txt/DCB_se1_ag1_m_01_1_part...,"Uh, I had /began before the street life/ for- ...",28.943,18,Male,data_processed_DCB/wav/DCB_se1_ag1_m_01_1_part...,play street lights for a swage 14 because of t...


In [19]:
print(len(audio_files))

4361


In [20]:
len(result[result.google_transcription.isna()])

0

In [21]:
result.to_csv(OUTPUT_PATH, index = False)